In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy.random as npr
import random

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from keras.optimizers import Adam
from keras_nlp.layers import PositionEmbedding

In [3]:
seed = 428

np.random.seed(seed)
tf.random.set_seed(seed)
random.seed(seed)

In [4]:
def bert_module(query, key, value, embed_dim, num_head, i):
    
    # Multi headed self-attention
    attention_output = layers.MultiHeadAttention(
        num_heads=num_head,
        key_dim=embed_dim // num_head,
        name="encoder_{}/multiheadattention".format(i)
    )(query, key, value, use_causal_mask=True)
    
    # Add & Normalize
    attention_output = layers.Add()([query, attention_output])  # Skip Connection
    attention_output = layers.LayerNormalization(epsilon=1e-6)(attention_output)
    
    # Feedforward network
    ff_net = keras.models.Sequential([
        layers.Dense(2 * embed_dim, activation='relu', name="encoder_{}/ffn_dense_1".format(i)),
        layers.Dense(embed_dim, name="encoder_{}/ffn_dense_2".format(i)),
    ])

    # Apply Feedforward network
    ffn_output = ff_net(attention_output)

    # Add & Normalize
    ffn_output = layers.Add()([attention_output, ffn_output])  # Skip Connection
    ffn_output = layers.LayerNormalization(epsilon=1e-6)(ffn_output)
    
    return ffn_output

In [5]:
def get_sinusoidal_embeddings(sequence_length, embedding_dim):
    position_enc = np.array([
        [pos / np.power(10000, 2. * i / embedding_dim) for i in range(embedding_dim)]
        if pos != 0 else np.zeros(embedding_dim)
        for pos in range(sequence_length)
    ])
    position_enc[1:, 0::2] = np.sin(position_enc[1:, 0::2])  # dim 2i
    position_enc[1:, 1::2] = np.cos(position_enc[1:, 1::2])  # dim 2i+1
    return tf.cast(position_enc, dtype=tf.float32)

In [6]:
N = 20 # vocab_size

vocabs = ['word_' + str(i) for i in range(N)]

vocab_map = {}
for i in range(len(vocabs)):
    vocab_map[vocabs[i]] = i
    
pairs = []

for i in vocabs:
    for j in vocabs:
        for k in vocabs:
            if i != j and i != k and j != k:
                pairs.append((i,j,k))
            
indicator = np.random.choice([0, 1], size=len(pairs), p=[0.5, 0.5])

pairs_train = [pairs[i] for i in range(len(indicator)) if indicator[i] == 1]
pairs_test = [pairs[i] for i in range(len(indicator)) if indicator[i] == 0]

In [7]:
sentences_train = []
sentences_number_train = []
sentences_test = []
sentences_number_test = []

for pair in pairs_train:
    sentences_train.append([pair[0], pair[1], pair[2], pair[0]])
    sentences_number_train.append([vocab_map[pair[0]], vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    
for pair in pairs_test:
    sentences_test.append([pair[0], pair[1], pair[2], pair[0]])
    sentences_number_test.append([vocab_map[pair[0]], vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    
x_masked_train = []
y_masked_labels_train = []
x_masked_test = []
y_masked_labels_test = []

for pair in pairs_train:
    x_masked_train.append([vocab_map[pair[0]], vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    y_masked_labels_train.append([vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    
for pair in pairs_test:
    x_masked_test.append([vocab_map[pair[0]], vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    y_masked_labels_test.append([vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    
x_masked_train = np.array(x_masked_train)
y_masked_labels_train = np.array(y_masked_labels_train)
x_masked_test = np.array(x_masked_test)
y_masked_labels_test = np.array(y_masked_labels_test)

perm = np.random.permutation(len(x_masked_train))
x_masked_train = x_masked_train[perm]
y_masked_labels_train = y_masked_labels_train[perm]

In [8]:
embed_dim = 10
num_head = 2

callback = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights = True)
inputs = layers.Input((x_masked_train.shape[1],), dtype=tf.int64)
word_embeddings = layers.Embedding(N, embed_dim, name="word_embedding")(inputs)
sinusoidal_embeddings = get_sinusoidal_embeddings(len(x_masked_train[0]), embed_dim)
encoder_output = word_embeddings + sinusoidal_embeddings

for i in range(5):
    encoder_output = bert_module(encoder_output, encoder_output, encoder_output, embed_dim, num_head, i)

encoder_output = keras.layers.Lambda(lambda x: x[:,:-1,:], name='slice')(encoder_output)
mlm_output = layers.Dense(N, name="mlm_cls", activation="softmax")(encoder_output)
mlm_model = keras.Model(inputs = inputs, outputs = mlm_output)
adam = Adam()
mlm_model.compile(loss='sparse_categorical_crossentropy', optimizer=adam)
history = mlm_model.fit(x_masked_train, y_masked_labels_train,
                        validation_split = 0.5, callbacks = [callback], 
                        epochs=2000, batch_size=5000, 
                        verbose=2)

2024-05-22 07:04:43.714790: W tensorflow/tsl/platform/default/dso_loader.cc:66] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-05-22 07:04:43.714826: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2024-05-22 07:04:43.714846: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (gl3035.arc-ts.umich.edu): /proc/driver/nvidia/version does not exist
2024-05-22 07:04:43.715024: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/2000
1/1 - 4s - loss: 3.2037 - val_loss: 3.1891 - 4s/epoch - 4s/step
Epoch 2/2000
1/1 - 0s - loss: 3.1765 - val_loss: 3.1696 - 100ms/epoch - 100ms/step
Epoch 3/2000
1/1 - 0s - loss: 3.1544 - val_loss: 3.1508 - 97ms/epoch - 97ms/step
Epoch 4/2000
1/1 - 0s - loss: 3.1341 - val_loss: 3.1334 - 102ms/epoch - 102ms/step
Epoch 5/2000
1/1 - 0s - loss: 3.1156 - val_loss: 3.1167 - 97ms/epoch - 97ms/step
Epoch 6/2000
1/1 - 0s - loss: 3.0982 - val_loss: 3.0996 - 97ms/epoch - 97ms/step
Epoch 7/2000
1/1 - 0s - loss: 3.0803 - val_loss: 3.0836 - 96ms/epoch - 96ms/step
Epoch 8/2000
1/1 - 0s - loss: 3.0632 - val_loss: 3.0682 - 97ms/epoch - 97ms/step
Epoch 9/2000
1/1 - 0s - loss: 3.0464 - val_loss: 3.0523 - 97ms/epoch - 97ms/step
Epoch 10/2000
1/1 - 0s - loss: 3.0296 - val_loss: 3.0360 - 98ms/epoch - 98ms/step
Epoch 11/2000
1/1 - 0s - loss: 3.0130 - val_loss: 3.0200 - 103ms/epoch - 103ms/step
Epoch 12/2000
1/1 - 0s - loss: 2.9967 - val_loss: 3.0047 - 94ms/epoch - 94ms/step
Epoch 13/2000
1/1 - 0s 

Epoch 101/2000
1/1 - 0s - loss: 2.4623 - val_loss: 2.4888 - 105ms/epoch - 105ms/step
Epoch 102/2000
1/1 - 0s - loss: 2.4577 - val_loss: 2.4849 - 97ms/epoch - 97ms/step
Epoch 103/2000
1/1 - 0s - loss: 2.4536 - val_loss: 2.4819 - 109ms/epoch - 109ms/step
Epoch 104/2000
1/1 - 0s - loss: 2.4506 - val_loss: 2.4780 - 101ms/epoch - 101ms/step
Epoch 105/2000
1/1 - 0s - loss: 2.4477 - val_loss: 2.4751 - 100ms/epoch - 100ms/step
Epoch 106/2000
1/1 - 0s - loss: 2.4446 - val_loss: 2.4706 - 100ms/epoch - 100ms/step
Epoch 107/2000
1/1 - 0s - loss: 2.4412 - val_loss: 2.4687 - 103ms/epoch - 103ms/step
Epoch 108/2000
1/1 - 0s - loss: 2.4390 - val_loss: 2.4658 - 104ms/epoch - 104ms/step
Epoch 109/2000
1/1 - 0s - loss: 2.4366 - val_loss: 2.4648 - 99ms/epoch - 99ms/step
Epoch 110/2000
1/1 - 0s - loss: 2.4357 - val_loss: 2.4613 - 96ms/epoch - 96ms/step
Epoch 111/2000
1/1 - 0s - loss: 2.4327 - val_loss: 2.4565 - 97ms/epoch - 97ms/step
Epoch 112/2000
1/1 - 0s - loss: 2.4269 - val_loss: 2.4494 - 98ms/epoch - 

Epoch 200/2000
1/1 - 0s - loss: 2.2347 - val_loss: 2.2549 - 81ms/epoch - 81ms/step
Epoch 201/2000
1/1 - 0s - loss: 2.2282 - val_loss: 2.2557 - 73ms/epoch - 73ms/step
Epoch 202/2000
1/1 - 0s - loss: 2.2313 - val_loss: 2.2497 - 83ms/epoch - 83ms/step
Epoch 203/2000
1/1 - 0s - loss: 2.2243 - val_loss: 2.2540 - 73ms/epoch - 73ms/step
Epoch 204/2000
1/1 - 0s - loss: 2.2269 - val_loss: 2.2474 - 82ms/epoch - 82ms/step
Epoch 205/2000
1/1 - 0s - loss: 2.2210 - val_loss: 2.2471 - 80ms/epoch - 80ms/step
Epoch 206/2000
1/1 - 0s - loss: 2.2221 - val_loss: 2.2441 - 79ms/epoch - 79ms/step
Epoch 207/2000
1/1 - 0s - loss: 2.2183 - val_loss: 2.2442 - 87ms/epoch - 87ms/step
Epoch 208/2000
1/1 - 0s - loss: 2.2180 - val_loss: 2.2400 - 89ms/epoch - 89ms/step
Epoch 209/2000
1/1 - 0s - loss: 2.2148 - val_loss: 2.2388 - 81ms/epoch - 81ms/step
Epoch 210/2000
1/1 - 0s - loss: 2.2143 - val_loss: 2.2375 - 80ms/epoch - 80ms/step
Epoch 211/2000
1/1 - 0s - loss: 2.2119 - val_loss: 2.2366 - 82ms/epoch - 82ms/step
Epoc

Epoch 299/2000
1/1 - 0s - loss: 2.1103 - val_loss: 2.1334 - 81ms/epoch - 81ms/step
Epoch 300/2000
1/1 - 0s - loss: 2.1087 - val_loss: 2.1328 - 88ms/epoch - 88ms/step
Epoch 301/2000
1/1 - 0s - loss: 2.1069 - val_loss: 2.1312 - 88ms/epoch - 88ms/step
Epoch 302/2000
1/1 - 0s - loss: 2.1057 - val_loss: 2.1303 - 85ms/epoch - 85ms/step
Epoch 303/2000
1/1 - 0s - loss: 2.1053 - val_loss: 2.1315 - 74ms/epoch - 74ms/step
Epoch 304/2000
1/1 - 0s - loss: 2.1054 - val_loss: 2.1300 - 81ms/epoch - 81ms/step
Epoch 305/2000
1/1 - 0s - loss: 2.1053 - val_loss: 2.1319 - 79ms/epoch - 79ms/step
Epoch 306/2000
1/1 - 0s - loss: 2.1051 - val_loss: 2.1284 - 91ms/epoch - 91ms/step
Epoch 307/2000
1/1 - 0s - loss: 2.1034 - val_loss: 2.1276 - 82ms/epoch - 82ms/step
Epoch 308/2000
1/1 - 0s - loss: 2.1015 - val_loss: 2.1260 - 82ms/epoch - 82ms/step
Epoch 309/2000
1/1 - 0s - loss: 2.1002 - val_loss: 2.1251 - 83ms/epoch - 83ms/step
Epoch 310/2000
1/1 - 0s - loss: 2.0997 - val_loss: 2.1263 - 76ms/epoch - 76ms/step
Epoc

Epoch 398/2000
1/1 - 0s - loss: 2.0495 - val_loss: 2.0769 - 84ms/epoch - 84ms/step
Epoch 399/2000
1/1 - 0s - loss: 2.0494 - val_loss: 2.0785 - 75ms/epoch - 75ms/step
Epoch 400/2000
1/1 - 0s - loss: 2.0490 - val_loss: 2.0762 - 83ms/epoch - 83ms/step
Epoch 401/2000
1/1 - 0s - loss: 2.0484 - val_loss: 2.0769 - 75ms/epoch - 75ms/step
Epoch 402/2000
1/1 - 0s - loss: 2.0475 - val_loss: 2.0751 - 81ms/epoch - 81ms/step
Epoch 403/2000
1/1 - 0s - loss: 2.0467 - val_loss: 2.0750 - 81ms/epoch - 81ms/step
Epoch 404/2000
1/1 - 0s - loss: 2.0462 - val_loss: 2.0750 - 81ms/epoch - 81ms/step
Epoch 405/2000
1/1 - 0s - loss: 2.0459 - val_loss: 2.0740 - 82ms/epoch - 82ms/step
Epoch 406/2000
1/1 - 0s - loss: 2.0458 - val_loss: 2.0752 - 76ms/epoch - 76ms/step
Epoch 407/2000
1/1 - 0s - loss: 2.0457 - val_loss: 2.0736 - 81ms/epoch - 81ms/step
Epoch 408/2000
1/1 - 0s - loss: 2.0456 - val_loss: 2.0750 - 74ms/epoch - 74ms/step
Epoch 409/2000
1/1 - 0s - loss: 2.0453 - val_loss: 2.0729 - 82ms/epoch - 82ms/step
Epoc

Epoch 497/2000
1/1 - 0s - loss: 2.0206 - val_loss: 2.0520 - 94ms/epoch - 94ms/step
Epoch 498/2000
1/1 - 0s - loss: 2.0201 - val_loss: 2.0528 - 85ms/epoch - 85ms/step
Epoch 499/2000
1/1 - 0s - loss: 2.0194 - val_loss: 2.0520 - 102ms/epoch - 102ms/step
Epoch 500/2000
1/1 - 0s - loss: 2.0189 - val_loss: 2.0514 - 91ms/epoch - 91ms/step
Epoch 501/2000
1/1 - 0s - loss: 2.0186 - val_loss: 2.0521 - 81ms/epoch - 81ms/step
Epoch 502/2000
1/1 - 0s - loss: 2.0186 - val_loss: 2.0511 - 91ms/epoch - 91ms/step
Epoch 503/2000
1/1 - 0s - loss: 2.0186 - val_loss: 2.0522 - 82ms/epoch - 82ms/step
Epoch 504/2000
1/1 - 0s - loss: 2.0185 - val_loss: 2.0508 - 87ms/epoch - 87ms/step
Epoch 505/2000
1/1 - 0s - loss: 2.0181 - val_loss: 2.0514 - 80ms/epoch - 80ms/step
Epoch 506/2000
1/1 - 0s - loss: 2.0177 - val_loss: 2.0504 - 91ms/epoch - 91ms/step
Epoch 507/2000
1/1 - 0s - loss: 2.0174 - val_loss: 2.0508 - 81ms/epoch - 81ms/step
Epoch 508/2000
1/1 - 0s - loss: 2.0172 - val_loss: 2.0508 - 82ms/epoch - 82ms/step
Ep

In [9]:
acc = []
prob = []
x_test_subset = x_masked_test[np.random.choice(x_masked_test.shape[0], size=1000, replace=False)]

for sentence_number in x_test_subset:
    temp = keras.backend.function(inputs = mlm_model.layers[0].input, outputs = mlm_model.layers[-1].output) \
        (np.array(sentence_number).reshape(1,len(sentence_number)))
    temp = temp[:,-1,:]
    acc.append(1 if temp.argmax() == sentence_number[-1] else 0)
    prob.append(temp[0][sentence_number[-1]])

In [10]:
(np.mean(acc), np.mean(prob))

(1.0, 0.9048236)

In [11]:
embed_dim = 100
num_head = 2

callback = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights = True)
inputs = layers.Input((x_masked_train.shape[1],), dtype=tf.int64)
word_embeddings = layers.Embedding(N, embed_dim, name="word_embedding")(inputs)
sinusoidal_embeddings = get_sinusoidal_embeddings(len(x_masked_train[0]), embed_dim)
encoder_output = word_embeddings + sinusoidal_embeddings

for i in range(5):
    encoder_output = bert_module(encoder_output, encoder_output, encoder_output, embed_dim, num_head, i)

encoder_output = keras.layers.Lambda(lambda x: x[:,:-1,:], name='slice')(encoder_output)
mlm_output = layers.Dense(N, name="mlm_cls", activation="softmax")(encoder_output)
mlm_model = keras.Model(inputs = inputs, outputs = mlm_output)
adam = Adam()
mlm_model.compile(loss='sparse_categorical_crossentropy', optimizer=adam)
history = mlm_model.fit(x_masked_train, y_masked_labels_train,
                        validation_split = 0.5, callbacks = [callback], 
                        epochs=2000, batch_size=5000, 
                        verbose=2)

Epoch 1/2000
1/1 - 5s - loss: 3.7975 - val_loss: 3.5023 - 5s/epoch - 5s/step
Epoch 2/2000
1/1 - 0s - loss: 3.4826 - val_loss: 3.3289 - 477ms/epoch - 477ms/step
Epoch 3/2000
1/1 - 1s - loss: 3.2997 - val_loss: 3.2003 - 560ms/epoch - 560ms/step
Epoch 4/2000
1/1 - 1s - loss: 3.1685 - val_loss: 3.1501 - 596ms/epoch - 596ms/step
Epoch 5/2000
1/1 - 1s - loss: 3.1184 - val_loss: 3.1181 - 558ms/epoch - 558ms/step
Epoch 6/2000
1/1 - 1s - loss: 3.0889 - val_loss: 3.0813 - 597ms/epoch - 597ms/step
Epoch 7/2000
1/1 - 1s - loss: 3.0556 - val_loss: 3.0442 - 582ms/epoch - 582ms/step
Epoch 8/2000
1/1 - 1s - loss: 3.0219 - val_loss: 3.0110 - 600ms/epoch - 600ms/step
Epoch 9/2000
1/1 - 1s - loss: 2.9908 - val_loss: 2.9813 - 629ms/epoch - 629ms/step
Epoch 10/2000
1/1 - 0s - loss: 2.9611 - val_loss: 2.9534 - 478ms/epoch - 478ms/step
Epoch 11/2000
1/1 - 1s - loss: 2.9311 - val_loss: 2.9291 - 552ms/epoch - 552ms/step
Epoch 12/2000
1/1 - 0s - loss: 2.9033 - val_loss: 2.9102 - 464ms/epoch - 464ms/step
Epoch 1

In [12]:
acc = []
prob = []
x_test_subset = x_masked_test[np.random.choice(x_masked_test.shape[0], size=1000, replace=False)]

for sentence_number in x_test_subset:
    temp = keras.backend.function(inputs = mlm_model.layers[0].input, outputs = mlm_model.layers[-1].output) \
        (np.array(sentence_number).reshape(1,len(sentence_number)))
    temp = temp[:,-1,:]
    acc.append(1 if temp.argmax() == sentence_number[-1] else 0)
    prob.append(temp[0][sentence_number[-1]])

In [13]:
(np.mean(acc), np.mean(prob))

(1.0, 0.98095894)